# Graded Challenge 3
Izzan Dienurrahman HCK-003

## Deskripsi 
Saya bekerja sebagai data analis di The Look yang merupakan platform e-commerce terbesar di planet mars dan saya bertugas untuk membuat laporan evaluasi aktivitas penjualan di platform tersebut.

## Objektif 
Meningkatkan revenue penjualan produk dari platform e-commerce The Look sebesar 10% pada tiap bulan-bulan berikutnya hingga akhir Q4.

## Problem Statement
- Mengevaluasi data historis revenue bulanan.
- Menentukan target kenaikan revenue yang wajar
- Menentukan target jumlah penjualan produk bulan-bulan berikutnya
- Mengetahui tren suatu produk & jumlah transaksi serta korelasi dari hal-hal yang berkaitan dengan revenue.
- Merencanakan strategi untuk kuartal berikutnya (Q4)

In [95]:
# import library yang dibutuhkan
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Authenticated') # print ketika auth sukses

project_id = "hacktiv8-izzan"
client = bigquery.Client(project=project_id) # connect ke google big query

Authenticated


1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [96]:
# jumlah transaksi tiap bulan selama Q1-Q3 tahun 2022 dengan status 'Complete'
query = '''SELECT 
COUNT(CASE WHEN month=1 THEN status ELSE NULL END) AS Jan,
COUNT(CASE WHEN month=2 THEN status ELSE NULL END) AS Feb,
COUNT(CASE WHEN month=3 THEN status ELSE NULL END) AS Mar,
COUNT(CASE WHEN month=4 THEN status ELSE NULL END) AS Apr,
COUNT(CASE WHEN month=5 THEN status ELSE NULL END) AS May,
COUNT(CASE WHEN month=6 THEN status ELSE NULL END) AS Jun,
COUNT(CASE WHEN month=7 THEN status ELSE NULL END) AS Jul,
COUNT(CASE WHEN month=8 THEN status ELSE NULL END) AS Aug,
COUNT(CASE WHEN month=9 THEN status ELSE NULL END) AS Sep
FROM (SELECT *,
EXTRACT (YEAR FROM ord_items.created_at) as year,
EXTRACT (MONTH FROM ord_items.created_at) as month
 FROM `bigquery-public-data.thelook_ecommerce.order_items` as ord_items) AS sub
WHERE status = 'Complete' AND year=2022'''
df_total_trans = client.query(query).to_dataframe()
df_total_trans.style.format("{:,}")

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep
0,"1,316","1,364","1,564","1,635","1,750","1,879","2,111","2,378","2,442"


- Secara umum terjadi peningkatan jumlah transaksi yang cukup konsisten tiap bulannya pada kuartal 1 hingga kuartal 3.

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [97]:
# total penjualan bulanan selama Q1-Q3 tahun 2022
query = '''SELECT 
SUM(CASE WHEN month=1 THEN sale_price ELSE NULL END) AS Jan,
SUM(CASE WHEN month=2 THEN sale_price ELSE NULL END) AS Feb,
SUM(CASE WHEN month=3 THEN sale_price ELSE NULL END) AS Mar,
SUM(CASE WHEN month=4 THEN sale_price ELSE NULL END) AS Apr,
SUM(CASE WHEN month=5 THEN sale_price ELSE NULL END) AS May,
SUM(CASE WHEN month=6 THEN sale_price ELSE NULL END) AS Jun,
SUM(CASE WHEN month=7 THEN sale_price ELSE NULL END) AS Jul,
SUM(CASE WHEN month=8 THEN sale_price ELSE NULL END) AS Aug,
SUM(CASE WHEN month=9 THEN sale_price ELSE NULL END) AS Sep
FROM (SELECT *,
EXTRACT (YEAR FROM ord_items.created_at) as year,
EXTRACT (MONTH FROM ord_items.created_at) as month
 FROM `bigquery-public-data.thelook_ecommerce.order_items` as ord_items) AS sub
WHERE status = 'Complete' AND year=2022'''
df_total_sales = client.query(query).to_dataframe()
df_total_sales.style.format("${:,.2f}")

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep
0,"$81,454.26","$85,810.48","$93,958.48","$105,278.45","$101,416.36","$108,924.85","$124,116.78","$134,899.61","$145,558.89"


- Total penjualan tiap bulannya mengalami peningkatan yang konsisten dan berbanding lurus dengan data jumlah transaksi sebelumnya.

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [98]:
# jumlah unique user yang melakukan transaksi tiap bulan selama Q1-Q3 tahun 2022
query = '''SELECT
COUNT (DISTINCT CASE WHEN month=1 THEN user_id ELSE NULL END) AS Jan,
COUNT (DISTINCT CASE WHEN month=2 THEN user_id ELSE NULL END) AS Feb,
COUNT (DISTINCT CASE WHEN month=3 THEN user_id ELSE NULL END) AS Mar,
COUNT (DISTINCT CASE WHEN month=4 THEN user_id ELSE NULL END) AS Apr,
COUNT (DISTINCT CASE WHEN month=5 THEN user_id ELSE NULL END) AS May,
COUNT (DISTINCT CASE WHEN month=6 THEN user_id ELSE NULL END) AS Jun,
COUNT (DISTINCT CASE WHEN month=7 THEN user_id ELSE NULL END) AS Jul,
COUNT (DISTINCT CASE WHEN month=8 THEN user_id ELSE NULL END) AS Aug,
COUNT (DISTINCT CASE WHEN month=9 THEN user_id ELSE NULL END) AS Sep
FROM (SELECT *,
EXTRACT (YEAR FROM ord_items.created_at) as year,
EXTRACT (MONTH FROM ord_items.created_at) as month
 FROM `bigquery-public-data.thelook_ecommerce.order_items` as ord_items
 JOIN `bigquery-public-data.thelook_ecommerce.inventory_items`as inv_items
ON ord_items.product_id = inv_items.product_id) AS sub
WHERE status = 'Complete' AND year=2022'''
df_total_user = client.query(query).to_dataframe()
df_total_user.style.format("{:,}")

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep
0,909,941,"1,074","1,150","1,252","1,338","1,462","1,640","1,710"


- Secara umum user yang melakukan transaksi mengalami peningkatan tiap bulannya. Sejauh ini, dapat dikatakan jumlah user yang melakukan transaksi berbanding lurus dengan banyaknya total transaksi dan juga total penjualan.

4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [108]:
# kategori produk yang paling banyak dibeli tiap bulan pada Q1-Q3 2022
query = '''
SELECT * 
FROM
(SELECT *,
RANK() OVER(partition by month order by sub2.total_sales desc) as rank_sales
FROM
(SELECT 
  month,
  product_category,
  COUNT(month) AS total_sales
FROM (SELECT *,
EXTRACT (YEAR FROM ord_items.created_at) as year,
EXTRACT (MONTH FROM ord_items.created_at) as month
 FROM `bigquery-public-data.thelook_ecommerce.order_items` as ord_items
 JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` as inv_items
ON ord_items.product_id = inv_items.product_id) AS sub
WHERE status = 'Complete' AND year=2022 AND month BETWEEN 1 AND 9
GROUP BY product_category,month) AS sub2
ORDER BY month) AS sub3
WHERE rank_sales =1
ORDER BY month
'''
df = client.query(query).to_dataframe()
df.style.format({"total_sales":"{:,}"})

,month,product_category,total_sales,rank_sales
0,1,Jeans,"2,020",1
1,2,Swim,"2,027",1
2,3,Tops & Tees,"2,419",1
3,4,Tops & Tees,"2,493",1
4,5,Intimates,"2,473",1
5,6,Jeans,"3,146",1
6,7,Jeans,"3,040",1
7,8,Shorts,"3,437",1
8,9,Jeans,"3,652",1


- Pada awal Q1 terlihat populernya penjualan Jeans, Swim, dan Top & tees
- Memasuki Q2 Tops & Tees, Intimates, dan Jeans menjadi kategori produk yang terlaris.
- Pada Q3 Jeans & Short menjadi kategori produk yang paling banyak dibeli.
- Jeans merupakan kategori terlaris.
- Peminat Jeans selalu ada sepanjang kuartal.

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [100]:
# kategori produk dengan pendapatan tertinggi tiap bulan pada Q1-Q3 2022
query = '''
SELECT *,
FROM
(SELECT *,
RANK() OVER(partition by month order by sub2.gross_revenue desc) as rank_revenue
FROM
(SELECT month,
product_category,
SUM(sale_price) AS gross_revenue
FROM
(SELECT product_category,sale_price,status,
EXTRACT (YEAR FROM ord_items.created_at) as year,
EXTRACT (MONTH FROM ord_items.created_at) as month
 FROM `bigquery-public-data.thelook_ecommerce.order_items` as ord_items
 JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` as inv_items
ON ord_items.product_id = inv_items.product_id) AS sub
WHERE status='Complete' AND year =2022 AND month BETWEEN 1 AND 9
GROUP BY month, product_category) AS sub2
ORDER BY month) as sub3
WHERE rank_revenue=1
'''
df = client.query(query).to_dataframe()
df.style.format({"gross_revenue":"$ {:,.2f}"})

,month,product_category,gross_revenue,rank_revenue
0,1,Outerwear & Coats,"$ 192,537.59",1
1,2,Jeans,"$ 217,275.20",1
2,3,Outerwear & Coats,"$ 218,551.28",1
3,4,Outerwear & Coats,"$ 296,134.54",1
4,5,Jeans,"$ 241,062.81",1
5,6,Jeans,"$ 315,775.35",1
6,7,Outerwear & Coats,"$ 311,862.72",1
7,8,Jeans,"$ 319,362.39",1
8,9,Outerwear & Coats,"$ 372,053.00",1


- Secara umum Jeans dan Outerwear & Coats menjadi kategori produk dengan pendapatan terbanyak tiap bulannya. Penulis menarik kesimpulan bahwa produk Jeans dan Outerwear & Coats merupakan pakaian yang paling diminati dan cocok digunakan sepanjang bulan pada Q1 hingga Q3.

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)


In [101]:
import pandas as pd # import library yang dibutuhkan

In [102]:
# menggabungkan kolom jumlah pelanggan, jumlah transaksi, dan jumlah revenue
columns = ['total_user','total_trans','total_sales']
df_corr = pd.concat([df_total_user.T,df_total_trans.T,df_total_sales.T],axis=1)
df_corr.columns=columns
df_corr

,total_user,total_trans,total_sales
Jan,909,1316,81454.259979
Feb,941,1364,85810.480045
Mar,1074,1564,93958.480139
Apr,1150,1635,105278.450055
May,1252,1750,101416.360103
Jun,1338,1879,108924.850134
Jul,1462,2111,124116.780211
Aug,1640,2378,134899.610057
Sep,1710,2442,145558.890296


In [103]:
df_corr.corr() # memanggil fungsi korelasi dan menampilkan nilainya.

,total_user,total_trans,total_sales
total_user,1.000000,0.997897,0.985321
total_trans,0.997897,1.000000,0.987978
total_sales,0.985321,0.987978,1.000000


- Dari data korelasi terlihat bahwa jumlah user(total_user) lebih berkaitan erat dengan jumlah transaksi(total_trans) dengan nilai korelasi sebesar 0.997 daripada total penjualan(total_sales) dengan nilai korelasi sebesar 0.985
- Kedua nilai merupakan positif dan sangat mendekati angka 1 sehingga dapat dikatakan hubungan mereka secara erat berbanding lurus, yang artinya kenaikan jumlah user mengakibatkan kenaikan pada jumlah transaksi dan juga total penjualan. Begitupun sebaliknya, penurunan jumlah user mengakibatkan penurunan jumlah transaksi dan juga total penjualannnya.

POIN ANALISIS
- Jumlah user, jumlah transaksi, dan total penjualan terus mengalami peningkatan dari awal Q1 hingga akhir Q3
- Kenaikan revenue bulanan kurang lebih sebesar 5-10%
- Kenaikan total penjualan bulanan sebesar 5-13%
- Jeans merupakan kategori produk terpopuler dan terlaris.
- Peminat Jeans selalu ada sepanjang kuartal
- Puncak peminat produk Swim terjadi pada bulan Februari
- Puncak peminat produk Tops & Tees terjadi pada bulan Maret-April
- Puncak peminat produk Shorts terjadi pada bulan Agustus
- Peningkatan jumlah penjualan sebesar kurang lebih 7-10% menghasilkan kenaikan revenue sebesar +-10%
- Melihat popularitas produk Jeans, penulis menyarankan untuk strategi penjualan pada kuartal 4, perbanyak model Jeans dan naikkan stocknya sebanyak 10% tiap bulannya. Selain itu, melihat spending habit warga planet Mars, sepertinya sudah cukup konsumtif. Namun untuk mendorong total sales yang lebih banyak, mungkin tim sales dan tim finance dapat mengatur cashback dan diskon dengan pertimbangan yang matang agar tren ini dapat dipertahankan. Jikalau hal ini dapat dipertahankan, revenue bulanan The Look e-commerce pada akhir Q4 diprediksikan dapat mencapai +- 213K USD.
  

POIN PERTANYAAN
1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
> - Ya, pertama karena spesifik yakni menaikkan revenue.
> - Kedua karena terukur dan wajar yakni ekspektasi peningkatan sebesar 10% setiap bulannya.
> - Ketiga karena peningkatan revenue dapat dicapai dengan meningkatnya jumlah transaksi yang konsisten sebesar 5-13%.
> - Keempat karena relevansi dari informasi tren barang dan jumlah user yang melakukan transaksi terus mengalami peningkatan serta korelasinya dengan revenue sangat erat dan berbanding lurus.
> - Kelima karena time-bound atau ada batas waktu yang sudah ditentukan dalam pencapaian target yakni pada akhir Q4.

2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?
>Rasanya kurang masuk akal, mengingat peningkatan jumlah user yang melakukan transaksi hanya sekitar 5-10% dari bulan ke bulan. Kalaupun mau dipaksakan, harus dibantu dengan peningkatan jumlah transaksi sekitar sebesar 70-80% dari user yang ada pada saat itu.

3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

In [161]:
# peningkatan jumlah user antar kuartal
query = '''
SELECT 
month,count(DISTINCT user_id) as jumlah_user,
(count(DISTINCT user_id)/LAG(count(DISTINCT user_id),1) over(order by month) -1)*100 as quarterly_percent_diff
FROM (SELECT *,
EXTRACT (YEAR FROM ord_items.created_at) as year,
EXTRACT (MONTH FROM ord_items.created_at) as month
 FROM `bigquery-public-data.thelook_ecommerce.order_items` as ord_items) AS sub
WHERE status = 'Complete' AND year=2022 AND month BETWEEN 1 AND 9
GROUP BY month
HAVING mod(month,3)=0 OR month=1
ORDER BY month
'''
df = client.query(query).to_dataframe()
df.style.format({"quarterly_percent_diff":"{:,.2f}%"})

,month,jumlah_user,quarterly_percent_diff
0,1,909,nan%
1,3,1074,18.15%
2,6,1338,24.58%
3,9,1710,27.80%


In [152]:
# jumlah transaksi Q1, Q2, Q3
trans_q1 = df_total_trans[['Jan','Feb','Mar']].T.sum()
trans_q2 = df_total_trans[['Apr','May','Jun']].T.sum()
trans_q3 = df_total_trans[['Jul','Aug','Sep']].T.sum()
columns=['Q1','Q2','Q3']
trans_q = pd.concat([trans_q1,trans_q2,trans_q3],axis=1)
trans_q.columns=columns
trans_q.style.format("{:,}")

,Q1,Q2,Q3
0,"4,244","5,264","6,931"


In [111]:
# total sales / GMV Q1-Q3 2022
df_total_sales.style.format("${:,.2f}")

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep
0,"$81,454.26","$85,810.48","$93,958.48","$105,278.45","$101,416.36","$108,924.85","$124,116.78","$134,899.61","$145,558.89"


In [137]:
# Akumulasi Gross Merchandise Value (GMV) pada Q1,Q2,Q3
gmv_q1 = df_total_sales[['Jan','Feb','Mar']].T.sum()
gmv_q2 = df_total_sales[['Apr','May','Jun']].T.sum()
gmv_q3 = df_total_sales[['Jul','Aug','Sep']].T.sum()
columns=['Q1','Q2','Q3']
gmv_q = pd.concat([gmv_q1,gmv_q2,gmv_q3],axis=1)
gmv_q.columns=columns
gmv_q.style.format("${:,.2f}")


,Q1,Q2,Q3
0,"$261,223.22","$315,619.66","$404,575.28"


- Dari aspek peningkatan jumlah user pada akhir Q1 mendapat 18.5% user baru dibanding awal Q1. Menjelang Akhir Q2, mendapat 24.58% user baru dibanding awal Q2. Menjelang Akhir Q3, mendapat 27.8% user baru dibanding awal Q3. Secara umum The Look e-commerce mendapatkan peningkatan jumlah user yang cukup stabil antar kuartalnya.
- Ditinjau dari aspek jumlah transaksi: menjelang akhir Q1, tercatat total jumlah transaksi sebanyak 4,244 Menjelang akhir Q2 terjadi kenaikan sebanyak 24% menjadi 5,264. Pada akhir Q3 terjadi peningkatan kembali sebanyak 31% menjadi 6,931 total transaksi dari jumlah transaksi yang tercatat pada Q2.
- Ditinjau dari aspek GMV, peningkatan Q2 terhadap Q1 sebesar 20.8% dan peningkatan dari Q3 terhadap Q2 sebesar 28%.
- Dari data diatas, dapat diprediksi GMV pada akhir kuartal 4 dapat mencapai peningkatan sebesar 20% yakni pada nominal $485,490
- Mengutip sumber dari [sini](https://www.ekrut.com/media/gmv-adalah),
>"*Investor umumnya akan tertarik dengan perusahaan yang tetap maju atau memiliki progress bisnis stabil, meski tidak selalu mencapai angka tertinggi."* 
- Jika ditinjau dari ketiga aspek diatas, ***The Look*** e-commerce sangat layak mendapatkan investor baru. Selain karena pertumbuhan yang stabil, peningkatan pertumbuhannya juga relatif besar.